In [ ]:
import pandas as pd
import string,nltk,json
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv("train.csv")
df.columns
df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [ ]:
df = df[['id','text','target']]

In [ ]:
df['text'].iloc[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
ps = PorterStemmer()
remove_punctuation = dict((ord(char), None) for char in string.punctuation)
def get_processed_tokens(text):
    lowers = text.lower()
    no_punctuation = lowers.translate(remove_punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    filtered = [word for word in tokens if not word in stopwords.words('english')]
    lemmatized_words = [ps.stem(item) for item in filtered ]
    Cleaned_text = ' '.join(lemmatized_words)
    return Cleaned_text
df['cleaned_text'] = df['text'].apply(lambda x: get_processed_tokens(x))

In [ ]:
df['cleaned_text'].iloc[0]

'deed reason earthquak may allah forgiv us'

In [ ]:
X_train = df['cleaned_text']
y_train = df['target']

test_data = pd.read_csv('test.csv')
test_data['cleaned_text'] = test_data['text'].apply(lambda x: get_processed_tokens(x))
X_test = test_data['cleaned_text']


# Create a CountVectorizer to convert text data into numerical features
# vectorizer = CountVectorizer()
# X_train_vectorized = vectorizer.fit_transform(X_train)
# X_test_vectorized = vectorizer.transform(X_test)

#TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_vectorized = tfidf_vectorizer.fit_transform(X_train)
X_test_vectorized = tfidf_vectorizer.transform(X_test)


rf_classifier = RandomForestClassifier(n_estimators=250)
rf_classifier.fit(X_train_vectorized, y_train)
y_pred = rf_classifier.predict(X_test_vectorized)
test_data['target'] = y_pred

final_data = test_data[['id','target']]

#saving the tested data with predicted labels to a CSV file
final_data.to_csv('results.csv', index=False)

# Gradient Boosting

In [ ]:
ps = PorterStemmer()
remove_punctuation = dict((ord(char), None) for char in string.punctuation)
def get_processed_words(text):
    lowers = text.lower()
    no_punctuation = lowers.translate(remove_punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    filtered = [word for word in tokens if not word in stopwords.words('english')]
    lemmatized_words = [ps.stem(item) for item in filtered ]
    return lemmatized_words

In [ ]:
news_corpus = pd.read_csv('train.csv')
news_corpus['cleaned_text'] = news_corpus['text'].apply(lambda x: get_processed_words(x))
X_train = news_corpus['cleaned_text']
y_train = news_corpus['target']

test_data = pd.read_csv('test.csv')
test_data['cleaned_text'] = test_data['text'].apply(lambda x: get_processed_words(x))
X_test = test_data['cleaned_text']


# Create a doc2Vectorizer to convert text data into numerical features ----> Used ChatGpt to understand this Concept
tagged_data_train = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(X_train)]
tagged_data_valid = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(X_test)]

# Initialize and train the Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
doc2vec_model.build_vocab(tagged_data_train)
doc2vec_model.train(tagged_data_train, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

X_train_doc2vec = [doc2vec_model.infer_vector(doc.words) for doc in tagged_data_train]
X_valid_doc2vec = [doc2vec_model.infer_vector(doc.words) for doc in tagged_data_valid]


In [ ]:
#I am getting max accuracy at the following parameter setting, The training accuracy is 100%

gb_classifier = GradientBoostingClassifier(n_estimators=500, min_samples_split=10, max_features=1000, random_state=42)
gb_classifier.fit(X_train_doc2vec, y_train)
Y_train_pred = gb_classifier.predict(X_train_doc2vec)

train_accuracy = accuracy_score(y_train, Y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")

Y_valid_pred = gb_classifier.predict(X_valid_doc2vec)
test_data['target'] = Y_valid_pred
final_data = test_data[['id','target']]

final_data.to_csv('labels_result.csv', index=False,header=False)

Training Accuracy: 0.90


# Neural Network

From section I get best results at learning_rate=0.001 and optimizer = Adam and Vectorizer as TF-IDF